In [ ]:
'''extractive QA
BERT - squad2.0'''

In [ ]:
!pip install transformers -q

In [ ]:
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

if not os.path.exists('/content/drive/MyDrive/Projects/BERT QA'):
    os.mkdir('/content/drive/MyDrive/Projects/BERT QA')